In [1]:
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
import pyarrow.fs as pafs
import requests
import io
import gzip
import os

In [7]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "C:/Users/Luka/Desktop/DESK/Studying/DE-zoomcamp/04-analytics-engineering-dbt/HW4/angular-sign-demo-405419-92acaf7265b7.json"
project_id = 'angular-sign-demo-405419'
bucket_name = 'de-zoomcamp-hw-3'

In [8]:
def csv_url_to_parquet_gcs(url, destination_blob_name):
    """Converts a CSV file from a URL to Parquet and uploads it directly to GCS using PyArrow."""
    
    taxi_dtypes = {
        'VendorID': 'Int64',
        'store_and_fwd_flag': 'str',
        'RatecodeID': 'Int64',
        'PULocationID': 'Int64',
        'DOLocationID': 'Int64',
        'passenger_count': 'Int64',
        'trip_distance': 'float64',
        'fare_amount': 'float64',
        'extra': 'float64',
        'mta_tax': 'float64',
        'tip_amount': 'float64',
        'tolls_amount': 'float64',
        'ehail_fee': 'float64',
        'improvement_surcharge': 'float64',
        'total_amount': 'float64',
        'payment_type': 'float64',
        'trip_type': 'float64',
        'congestion_surcharge': 'float64'
    }

    parse_dates = ['tpep_pickup_datetime', 'tpep_dropoff_datetime']
    
    df = pd.read_csv(url, sep=',', compression='gzip', dtype=taxi_dtypes, parse_dates=parse_dates)
    
    df['tpep_pickup_datetime'] = pd.to_datetime(df['tpep_pickup_datetime'])
    df['tpep_dropoff_datetime'] = pd.to_datetime(df['tpep_dropoff_datetime'])
    
    table = pa.Table.from_pandas(df)
    gcs_filesystem = pafs.GcsFileSystem()
    gcs_path = f"{bucket_name}/{destination_blob_name}"
    
    with gcs_filesystem.open_output_stream(gcs_path) as out_stream:
        pq.write_table(table, out_stream)
    
    print(f"Uploaded {url} as {destination_blob_name} to {bucket_name}")

In [11]:
years_to_download = [2019, 2020]
months_to_download = range(1, 13)

for year in years_to_download:
    for month in months_to_download:
        month_str = str(month)
        if len(month_str) == 1:
            month_str = '0' + month_str
        
        url = f'https://github.com/DataTalksClub/nyc-tlc-data/releases/download/yellow/yellow_tripdata_{year}-{month_str}.csv.gz'
        name = f'yellow_tripdata/{year}/yellow_tripdata_{year}-{month_str}.parquet'
        csv_url_to_parquet_gcs(url, name)

Uploaded https://github.com/DataTalksClub/nyc-tlc-data/releases/download/yellow/yellow_tripdata_2019-01.csv.gz as yellow_tripdata/2019/yellow_tripdata_2019-01.parquet to de-zoomcamp-hw-3
Uploaded https://github.com/DataTalksClub/nyc-tlc-data/releases/download/yellow/yellow_tripdata_2019-02.csv.gz as yellow_tripdata/2019/yellow_tripdata_2019-02.parquet to de-zoomcamp-hw-3
Uploaded https://github.com/DataTalksClub/nyc-tlc-data/releases/download/yellow/yellow_tripdata_2019-03.csv.gz as yellow_tripdata/2019/yellow_tripdata_2019-03.parquet to de-zoomcamp-hw-3
Uploaded https://github.com/DataTalksClub/nyc-tlc-data/releases/download/yellow/yellow_tripdata_2019-04.csv.gz as yellow_tripdata/2019/yellow_tripdata_2019-04.parquet to de-zoomcamp-hw-3
Uploaded https://github.com/DataTalksClub/nyc-tlc-data/releases/download/yellow/yellow_tripdata_2019-05.csv.gz as yellow_tripdata/2019/yellow_tripdata_2019-05.parquet to de-zoomcamp-hw-3
Uploaded https://github.com/DataTalksClub/nyc-tlc-data/releases/d

KeyboardInterrupt: 